In [14]:
import numpy as np
import pandas as pd
import re    #Regular expression library
import string
import nltk
from collections import Counter
vocab=Counter()
import pickle

## Text Preprocessing

In [5]:
def removePunc(text):
    for punc in string.punctuation:
        text=text.replace(punc,' ')
    return text

In [6]:
with open('../static/model/corpora/stopwords/english','r') as stp:  #get stopwords into a list
    sw=stp.read().splitlines()

In [7]:
from nltk.stem import PorterStemmer
st=PorterStemmer()

In [11]:
def textPreprocessing(txt):
    data=pd.DataFrame([txt],columns=['tweet'])    
    data['tweet']=data['tweet'].apply(lambda x: " " .join(x.lower() for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: " ".join(re.sub(r'https?://\S+', '', word) for word in x.split()))
    data['tweet'] = data['tweet'].apply(removePunc)
    data['tweet'] = data['tweet'].str.replace('\d+','',regex=True)
    data['tweet']=data['tweet'].apply(lambda x: " " .join(x.lower() for x in x.split() if x not in sw)) 
    data['tweet']=data['tweet'].apply(lambda x: " " .join(st.stem(x) for x in x.split()))

    return data['tweet']

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
C:\Users\User\AppData\Local\Temp\ipykernel_15604\361471222.py:6: SyntaxWarning: invalid escape sequence '\d'
  data['tweet'] = data['tweet'].str.replace('\d+','',regex=True)


In [12]:
txt=" Good product , I like it"
textPreprocessing(txt)

0    good product like
Name: tweet, dtype: object

In [15]:
#Load model

with open('../static/model/model.pickle','rb') as model:
    model=pickle.load(model)

In [16]:
#Get vocabulary

vocab=pd.read_csv('../static/model/vocabulary.txt',header=None)
token=vocab[0].tolist()

In [17]:
def vectorizer(dataset, vocabulary):
    vectorized_lst=[]  #create empty list
    for sentence in dataset:  #go sentence by sentence in x
        sentence_lst=np.zeros(len(vocabulary))   #initialize the 0 list with respect to vocab size
        for i in range(len(vocabulary)):    # go word by word in sentece in vocab size
            if vocabulary[i] in sentence.split():  #if found vocab's word then replace 0 value by 1
                sentence_lst[i] = 1
        vectorized_lst.append(sentence_lst)   #append into another list as a list 
    vectorized_lst_new=np.asarray(vectorized_lst, dtype=np.float32)   #create fully completed list into np array
    return vectorized_lst_new

In [24]:
txt="very Good product, nice and awesome"
preprocessed_txt=textPreprocessing(txt)
vectorized_txt=vectorizer(preprocessed_txt,token)
pred=getPrediction(vectorized_txt)
pred

'Positive'

In [26]:
def getPrediction(vectorized_txt):
    pred=model.predict(vectorized_txt)
    if pred==0:
        return "Positive"
    else:
        return "Negative"
    